Plotting the results of the scalability experiments.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

# All groups of plots I want to plot:
configs_k_anonymity = ['5000000', '7500000', '10000000', '12500000', '15000000', '17500000', '20000000', '22500000', '25000000', '35000000', '45000000', '55000000', '65000000', '75000000', '85000000']

In [ ]:
def is_rightmost(subplot_code):
    # Extract the number of columns from the subplot code
    num_cols = (subplot_code // 10) % 10

    # Extract the subplot index
    index = subplot_code % 100

    return index % num_cols == 0

def is_leftmost(subplot_code):
    # Extract the number of columns from the subplot code
    num_cols = (subplot_code // 10) % 10

    # Extract the subplot index
    index = subplot_code % 100

    return index % num_cols == 1

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Get the tab10 colormap
tab10 = plt.cm.get_cmap('tab10', 10)

# Extract the colors from the colormap
colors = [tab10(i) for i in range(tab10.N)]

# Convert the RGBA colors to HEX format
hex_colors = [mcolors.rgb2hex(color[:3]) for color in colors]

print(hex_colors)

In [ ]:
import seaborn as sns
import os

color_map = ['plum','skyblue', 'red', 'green', 'royalblue', 'cyan', 'orange']

print(hex_colors)
color_map = ['tab:blue', 'tab:green', 'tab:pink', 'tab:cyan']



def plot_results(n, m, fig_size_x, fig_size_y, steps, filename, local):
    plt.rcParams.update({    'font.size': 15,
                             'axes.titlesize': 15  # Ensure the title size matches other text
                             })
    fig, axes = plt.subplots(n, m, figsize=(fig_size_x, fig_size_y))
    max_memory = 0
    time_max = 0;
    for step in steps:
        #memory = df_mem.loc[df_mem.Config == config]
        #memory = memory.loc[df_mem.Local == local]["Memory"]/1000000.0
        time = df.loc[df.Number == step]
        time = time.loc[df.Local == local]["Time"]/1000.0
        timePostproces = df.loc[df.Number == step]
        timePostproces = timePostproces.loc[df.Local == local]["TimePostprocess"]/1000.0
        timeQuality = df.loc[df.Config == step]
        timeQuality = timeQuality.loc[df.Local == local]["TimeQuality"]/1000.0
        time_curr = time.max() + timePostproces.max() + timeQuality.max()
        if time_curr > time_max:
            time_max = time_curr
        #if memory.max() > max_memory:
        #    max_memory = memory.max()
    i = 0
    for ax1 in axes.ravel():
        config = steps[i]
        threads = df.loc[df.Number == config]
        threads = threads.loc[df.Local == local]["Threads"]
        #memory = df_mem.loc[df_mem.Config == config]
        #memory = memory.loc[df_mem.Local == local]["Memory"]/1000000.0
        timePrepare = df.loc[df.Number == config]
        timePrepare = timePrepare.loc[df.Local == local]["TimePrepare"]/1000.0
        timeAnonymize = df.loc[df.Number == config]
        timeAnonymize = timeAnonymize.loc[df.Local == local]["TimeAnonymize"]/1000.0
        timeGlobalTransform = df.loc[df.Number == config]
        timeGlobalTransform = timeGlobalTransform.loc[df.Local == local]["TimeGlobalTransform"]/1000.0
        timePartitionByClass = df.loc[df.Number == config]
        timePartitionByClass = timePartitionByClass.loc[df.Local == local]["TimePartitionByClass"]/1000.0
        timeSuppress = df.loc[df.Number == config]
        timeSuppress = timeSuppress.loc[df.Local == local]["TimeSuppress"]/1000.0
        timePostproces = df.loc[df.Number == config]
        timePostproces = timePostproces.loc[df.Local == local]["TimePostprocess"]/1000.0
        timeQuality = df.loc[df.Number == config]
        timeQuality = timeQuality.loc[df.Local == local]["TimeQuality"]/1000.0
        granularity = df.loc[df.Number == config]
        granularity = granularity.loc[df.Local == local]["Granularity"]
        timeAll = timePrepare + timeAnonymize + timeGlobalTransform + timePartitionByClass + timeSuppress + timeQuality + timePostproces
        print(timeAll)
        optimal_time = timeAll.iloc[0]
        time_saved_percent = ((optimal_time - timeAll) / optimal_time) * 100
        optimal_utility = granularity.iloc[0]
        percent_difference_utility = ((optimal_utility - granularity) / optimal_utility) * 100
        #optimal_memory = memory.iloc[0]
        #percent_difference_memory = (memory / optimal_memory) * 100
        df_combined = pd.DataFrame({
            'Threads': threads,
            'Time': timeAll,
            'Percent Time Saved': time_saved_percent,
            'Granularity': granularity,
            'Percent Difference utility:': percent_difference_utility,
        #    'Memory': memory,
        #    'Percent Difference Memory': percent_difference_memory
        })
        df_combined.to_csv(filename[:-4]+config+".csv", index=False, sep=';')
        print(df_combined)
        # Find the index of the max percent difference
        max_diff_index = percent_difference_utility.idxmax()
        # Extract corresponding values
        max_percent_diff_value = percent_difference_utility[max_diff_index]
        corresponding_granularity = granularity[max_diff_index]
        corresponding_time = timeAll[max_diff_index]
        corresponding_time_saved_percent = time_saved_percent[max_diff_index]
        corresponding_threads = threads[max_diff_index]
        print(f"Maximum Percent Difference: {max_percent_diff_value}%")
        print(f"Corresponding Utility: {corresponding_granularity}")
        print(f"Corresponding Time: {corresponding_time}")
        print(f"Corresponding Time saved percent: {corresponding_time_saved_percent}")
        print(f"Corresponding Threads: {corresponding_threads}")
        ax1.set_facecolor('w')
        ax2 = ax1.twinx()
        ax3 = ax1.twinx()
        ax2.set_ylim(0, 100)
        ax1.set_ylim(0, time_max)
        ax1.stackplot(threads, timePrepare + timeAnonymize, timeGlobalTransform, timePartitionByClass + timeSuppress, timeQuality + timePostproces, labels=["(1) Partitioning + Anonymization", "(2) Harmonization", "(3) Compliance checking", "(4) Finalization",], colors = color_map, edgecolor = 'black', linewidths = 0)
        ax1.margins(x=0) #remove space between graph and y-axis
        ax1.set_xticks([1, 10, 20, 30, 40, 50, 60])
        ax1.set_ylabel("Time [s]")
        ax1.set_xlabel("Threads")
        ax1.grid(None)
        ax2.grid(None)
        ax3.grid(None)
        ax2.plot(threads, granularity*100.0, label="Utility [%]", color="k", marker=".", markersize=0.3)
        #ax3.plot(threads, memory, label="Memory [MB]", color="coral", marker=".", markersize=0.3)
        ax2.set_ylabel("Utility [%]")
        ax3.spines['right'].set_position(('axes', 1.25))
        ax3.set_ylabel("Memory [MB]")
        #print(memory.max())
        print(granularity.max())
        ax3.set_ylim(0, max_memory * 1.1)
        ax3.spines['right'].set_color('coral')
        ax3.tick_params(axis='y', colors='coral')
        ax3.yaxis.label.set_color('coral')
        if not is_rightmost(ax1, axes): # checks not rightmost
            ax2.set_ylabel(None)
            ax2.yaxis.set_ticks([])
            #ax2.spines[['right']].set_visible(False)
            ax3.set_ylabel(None)
            ax3.yaxis.set_ticks([])
            ax3.spines[['right']].set_visible(False)
        if not is_leftmost(ax1, axes): # checks not leftmost
            ax1.set_ylabel(None)
            ax1.yaxis.set_ticks([])
            ax1.spines[['right']].set_visible(False)
        handles, labels = ax1.get_legend_handles_labels()
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles3, labels3 = ax3.get_legend_handles_labels()
        if local:
            ax1.set_title(config + " (local)")
        else:
            ax1.set_title(config)
        ax3.spines['bottom'].set_color('black')
        ax3.spines['top'].set_color('black')
        ax2.spines['right'].set_color('black')
        ax3.spines['left'].set_color('black')
        i = i + 1
        fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.01), fancybox=False, shadow=False, ncol=2, facecolor='white', edgecolor='black')
        #fig.legend(handles2+handles3, labels2+labels3, loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=False, shadow=False, ncol=2, facecolor='white', edgecolor='black')
    fig.tight_layout()
    if os.path.isfile(filename):
        os.remove(filename)   # Opt.: os.system("rm "+strFile)
    fig.savefig(filename, format='png', bbox_inches='tight', dpi=300)
    #plt.tight_layout()
    #plt.savefig(filename, format='png')
    plt.show()


In [ ]:

# All groups of plots I want to plot:
steps = [5000000, 7500000, 10000000, 12500000, 15000000, 17500000, 20000000, 22500000, 25000000, 35000000, 45000000, 55000000, 65000000, 75000000, 85000000]

In [ ]:
import pandas as pd
df = pd.read_csv('C:/Users/halilovm/Desktop/ARX_DISTRIBUTED_RESULTS/results/result_scalability_04_12.csv', sep=";")
df['Number'] = df['Dataset'].str.extract('(\d+)').astype(int)
df


In [ ]:
df['Local'].unique()

In [ ]:
df = df[df['Local'] == True]
df

In [ ]:
# Extract dataset size from 'Dataset' column
df['Dataset size'] = df['Dataset'].str.extract('(\d+)').astype(int)

# Pivot the table to get Time values for each number of threads
df_pivot = df.pivot(index='Dataset size', columns='Threads', values='Time')

# Select the necessary columns
df_transformed = df_pivot[[1, 2,  12, 64]].copy()  # Adjust this to include any other threads number you have
df_transformed.reset_index(inplace=True)

# Rename columns
df_transformed.columns = ['Dataset size', '1', '2', '12', '64']

print(df_transformed)


In [ ]:
# Extract dataset size from 'Dataset' column
df['Dataset size'] = df['Dataset'].str.extract('(\d+)').astype(int)

# Pivot the table to get Time values for each number of threads
df_pivot = df.pivot(index='Dataset size', columns='Threads', values='Granularity')

# Select the necessary columns
df_transformed_granularity = df_pivot[[1, 2, 12, 64]].copy()  # Adjust this to include any other threads number you have
df_transformed_granularity.reset_index(inplace=True)

# Rename columns
df_transformed_granularity.columns = ['Dataset size', '1', '2', '12', '64']

print(df_transformed_granularity)


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({    'font.size': 15,
                         'axes.titlesize': 15  # Ensure the title size matches other text
                         })
x_ticks = [5000000, 10000000, 15000000, 20000000, 25000000, 35000000, 45000000, 55000000, 65000000, 75000000, 85000000]

granularity_df = df_transformed_granularity
time_df = df_transformed

fig, ax1 = plt.subplots(figsize=(10, 6))
fig.patch.set_facecolor('white')
ax1.set_facecolor('white')

colors = ['b', 'g', 'r', 'cyan']
threads = ['1', '2', '12', '64']

# Plot granularity on primary y-axis
for i, thread in enumerate(threads):
    ax1.plot(granularity_df['Dataset size'], granularity_df[thread], '-o', label=f'Utility {thread}', color=colors[i])

ax1.set_ylabel('Utility')
ax1.set_ylim([0.7, 1])
ax1.tick_params('y')
ax1.set_xlabel('Dataset Size')
ax1.set_xticks(x_ticks)
ax1.set_yscale('linear')

# Explicitly set x-axis tick marks to be visible and inside
ax1.tick_params(axis='x', which='both', direction='out', length=6, bottom=True)
x_tick_labels = [x // 1000000 for x in x_ticks]

ax1.set_xticklabels(x_tick_labels )

ax1.set_xlabel('Dataset size (in millions of records)')

ax1.margins(x=0) #remove space between graph and y-axis


ax1.grid(None)
# Display legend and show plot
handles, labels = ax1.get_legend_handles_labels()
from matplotlib.lines import Line2D
utility_handle = Line2D([], [], color='grey', marker='o', linestyle='-', label='Utility')
time_handle = Line2D([], [], color='grey', marker='o', linestyle='--', label='Utility')
invisible_handle = Line2D([0], [0], alpha=0, color='none', label='Invisible Item')

fig.legend(handles, ['1 Thread', '2 Threads', '12 Threads', '64 Threads'], loc='upper center', bbox_to_anchor=(0.5, 0.01), fancybox=False, shadow=False, ncol=4, facecolor='white', edgecolor='black')
print(handles)
ax1.spines['bottom'].set_color('black')
ax1.spines['top'].set_color('black')
ax1.spines['right'].set_color('black')
ax1.spines['left'].set_color('black')
plt.title("Scalability experiment")
fig.tight_layout()
if os.path.isfile("images/scalability.png"):
    os.remove("images/scalability.png")   # Opt.: os.system("rm "+strFile)
fig.savefig("images/scalability.png", format='png', bbox_inches='tight', dpi=300)
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({    'font.size': 15,
                         'axes.titlesize': 15  # Ensure the title size matches other text
                         })
x_ticks = [5000000, 10000000, 15000000, 20000000, 25000000, 35000000, 45000000, 55000000, 65000000, 75000000, 85000000]

granularity_df = df_transformed_granularity
time_df = df_transformed

fig, ax1 = plt.subplots(figsize=(10, 6))
fig.patch.set_facecolor('white')
ax1.set_facecolor('white')

colors = ['b', 'g', 'r', 'cyan']
threads = ['1', '2', '12', '64']

# Plot granularity on primary y-axis
for i, thread in enumerate(threads):
    ax1.plot(time_df['Dataset size'], time_df[thread]/1000, '--o', label=f'Time {thread}', color=colors[i])

ax1.set_ylabel('Utility')
ax1.set_ylim(bottom=0)
ax1.tick_params('y')
ax1.set_xlabel('Dataset Size')
ax1.set_xticks(x_ticks)
ax1.set_yscale('linear')

# Plot time on secondary y-axis
ax1.set_yscale('linear')

# Explicitly set x-axis tick marks to be visible and inside
x_tick_labels = [x // 1000000 for x in x_ticks]

print(x_tick_labels)
ax1.set_xticklabels(x_tick_labels)


ax1.set_ylabel('Time [s]')
ax1.set_xlabel('Dataset size (in millions of records)')

ax1.margins(x=0.005) #remove space between graph and y-axis

ax1.set_ylim(bottom=0) #remove space between graph and y-axis

ax1.grid(None)
# Display legend and show plot
handles, labels = ax1.get_legend_handles_labels()
from matplotlib.lines import Line2D
utility_handle = Line2D([], [], color='grey', marker='o', linestyle='-', label='Utility')
time_handle = Line2D([], [], color='grey', marker='o', linestyle='--', label='Utility')
invisible_handle = Line2D([0], [0], alpha=0, color='none', label='Invisible Item')


fig.legend(handles, ['1 Thread', '2 Threads', '12 Threads', '64 Threads'], loc='upper center', bbox_to_anchor=(0.5, 0.01), fancybox=False, shadow=False, ncol=4, facecolor='white', edgecolor='black')
print(handles)
ax1.spines['bottom'].set_color('black')
ax1.spines['top'].set_color('black')
ax1.spines['right'].set_color('black')
ax1.spines['left'].set_color('black')
plt.title("Scalability experiment")
fig.tight_layout()
if os.path.isfile("images/scalability-time.png"):
    os.remove("images/scalability-time.png")   # Opt.: os.system("rm "+strFile)
fig.savefig("images/scalability-time.png", format='png', bbox_inches='tight', dpi=300)
plt.show()

